In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle



In [3]:
data=pd.read_csv('churn_modelling_demo.csv')
# data.head()

## preprocessing removing the data
data = data.drop(['CustomerID'], axis=1)


In [7]:
## Encode categorical variables
label_gender_encoder=LabelEncoder()
data['Gender'] = label_gender_encoder.fit_transform(data['Gender']) 

## for geography
onehotencoder=OneHotEncoder()
geo_encode=onehotencoder.fit_transform(data[['Geography']]).toarray()
geo_encode_df=pd.DataFrame(geo_encode, columns=onehotencoder.get_feature_names_out(['Geography']))


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,0.0,0.0,1.0
4,0.0,1.0,0.0
...,...,...,...
9995,0.0,1.0,0.0
9996,0.0,1.0,0.0
9997,0.0,0.0,1.0
9998,0.0,0.0,1.0


In [8]:
#merge the changed data
data= pd.concat([data.drop('Geography', axis=1), geo_encode_df], axis=1)
data.head()

,Age,Gender,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,56,0,3,22739.93,1,1,0,138256.47,0,0.0,1.0,0.0
1,69,0,9,194192.10,2,0,0,131653.33,0,1.0,0.0,0.0
2,46,1,5,47334.24,3,0,0,15251.37,1,0.0,1.0,0.0
3,32,1,4,181424.00,3,0,1,49234.71,1,0.0,0.0,1.0
4,60,1,8,110319.02,4,1,1,62464.12,1,0.0,1.0,0.0


In [9]:
## split features into dependent and independent features
## independent variable
X = data.drop('EstimatedSalary', axis=1)
##dependent variable
Y = data['EstimatedSalary']


In [10]:
## split the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=55)

In [11]:
## Scaler feature
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [12]:
## save encoder and scaler files
with open('label_encoder_gender_reg.pkl','wb') as file:
    pickle.dump(label_gender_encoder, file)
with open('onehot_geography_encoder_reg.pkl', 'wb') as file:
    pickle.dump(onehotencoder, file)
with open('scaler_reg.pkl', 'wb') as file:
    pickle.dump(scaler, file)


Train the ANN with regression testing

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [16]:
#### Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) #default activation function for regression --> linear regression
])


## compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

## model summary
model.summary()

e:\codes\ai\project_1\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

Train the model with logs


In [17]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

## set up TensorBoard
log_dir="regressionlogs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

## set up the Early stopping
early_stopping_callbacks= EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [18]:
## Train the model
history=model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=100,
    callbacks=[early_stopping_callbacks, tensorboard_callback]
)

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - loss: 81177.8984 - mae: 81177.8984 - val_loss: 79192.3281 - val_mae: 79192.3281
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 81761.3359 - mae: 81761.3359 - val_loss: 77958.5938 - val_mae: 77958.5938
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 79411.5391 - mae: 79411.5391 - val_loss: 74610.1719 - val_mae: 74610.1719
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 75601.3125 - mae: 75601.3125 - val_loss: 68343.9141 - val_mae: 68343.9141
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 68248.8984 - mae: 68248.8984 - val_loss: 59891.6016 - val_mae: 59891.6016
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 59336.1250 - mae: 59336.1250 - val_loss: 51176.3477 - val_mae: 51176.3477
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 51848.8789 - mae: 51848.8789 - val_loss: 43880.0977 - val_mae: 43880.0977
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/st

In [19]:
%load_ext tensorboard

In [21]:
%tensorboard --logdir regressionlogs/fit

In [25]:
## Evalute model on the test data
test_loss,test_mae=model.evaluate(X_test, Y_test)
print(f'Test MAE : {test_mae}')

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 35321.6484 - mae: 35321.6484 
Test MAE : 35007.65234375


In [26]:
model.save('regression_model.h5')